---
# 2. Appropriation du cadre conceptuel

Dans cette première partie, vous devez vous approprier la problématique théorique et clarifier les principaux concepts qui guideront vos analyses.

## Question 1

Expliquez, ce que l’on entend par « communauté hybride » dans ce TP. Distinguez clairement les acteurs humains et les agents autonomes. Discutez, en vous appuyant sur le texte de cadrage, pourquoi la simple augmentation de la connectivité apparente (plus de liens, plus d’interactions) ne garantit pas une cohésion « profonde » de la communauté.

---
*Réponse*:

Une communauté hybride est une communauté qui contient à la fois des humains et des agents autonômes. Un agent autonôme est un agent non-humain allimenté par l'intelligence artificielle (en particulier les LLM ou modèles de langage naturel).

Comme les agents autonômes ne sont pas des humains, les liens créer par ces agent ne sont pas indicatif d'un lien entre deux personnes. Ça indique que les comptes interagissent entre eux mais pas qu'il y a un sens d'inclusivité dans une communauté. Un agent autonôme garanti l'engagement sur Twitter/X mais ne garantie pas la connection humaine du tout.


## Question 2

Définissez précisément, avec vos propres mots, les notions suivantes, en les reliant explicitement à des métriques de réseau que vous pourrez calculer plus tard :

* connectivité de surface
* cohésion profonde
* centralisation de l’attention
* fragmentation en sous-communautés
* polarisation

---
*Réponse:*

* Connectivité de surface: Nombre d'arretes par noeud, densité du reseau.
* Cohésion profonde: Coefficient de clustering, modularité, centralité de proximité.
* Centralisation de l’attention: centralité du degré, en particulier la distribution des degrés entrants.
* Fragmentation en sous-communautés: le nombre et la taille des sous-communautés.
* Polarisation: rapport entre les liens intracommunautaires et intercommunautaires; indique les chambres d'echo.

## Question 3

Présentez brièvement les trois mécanismes attribués aux agents autonomes dans le projet de recherche :

* imitation linguistique
* amplification sélective
* perturbation de la formation des liens

Pour chacun, formulez en une phrase la manière dont il pourrait, en principe, affecter la structure du réseau (par exemple en termes de centralisation ou de fragmentation).

---
*Réponse:*
* Imitation linguistique: Agents qui mimiquent le langage humain ==> augmentation de cohésion possible dans des clusters.
  Chaque agent essaie d'être le plus "humain" possible pour pouvoir intégrer la communauté efficacement.
* Amplification sélective: Agents qui interagissent d'une manière préférentielle avec certain noeuds ==> augmentation de la centralisation du degré
  Si ces agents interagissent avec un noeud plus souvent qu'avec d'autres noeuds, ça va augmenter la degré entrant de ce noeud préferé, et ainsi amméliorera son classement de centralité du degré.
* Perturbation de la formation des liens: Agents qui interviennent avec des interactions organiques et naturelles==> augmentation de fragmentation des communautés possible.

## Question 4

Reformulez en langage simple les trois hypothèses suivantes, en expliquant ce qu’elles impliquent concrètement pour le réseau et les comportements observables :

* **H1** : plus un agent autonome est intégré dans la communauté, plus il reçoit d’engagement (réponses, mentions).
* **H2** : l’activité globale des agents tend à augmenter la centralisation de l’attention autour de quelques nœuds tout en accroissant la fragmentation du réseau en sous-communautés.
* **H3** : la relation entre la part d’interactions d’un utilisateur humain impliquant des agents et son engagement global suit une forme non linéaire, avec un niveau intermédiaire d’exposition associé au maximum d’engagement.

Vous justifierez pour chaque hypothèse pourquoi elle est plausible au regard du texte de cadrage.

---
*Réponse:*


---
# 3. Construction du graphe et statistiques descriptives (socle commun)

Dans cette partie, vous construisez le graphe à partir des fichiers bruts et vous produisez des premiers résultats descriptifs.

## Question 5

Décrivez précisément le pipeline de construction de votre graphe d’interactions. Indiquez les variables utilisées (identifiants source et cible, type d’interaction, date, poids éventuel), le type de graphe retenu (orienté ou non, pondéré ou non), la fenêtre temporelle choisie (par exemple un mois ou une semaine) et les règles de filtrage appliquées (par exemple suppression des arêtes très rares). Justifiez vos choix par rapport à la problématique de cohésion communautaire.

In [1]:
# Importing Libraries

import pandas as pd
import igraph as ig
import numpy as np

In [2]:
# Importing Config Variables for nodes.csv and edges.csv (file paths)
from scripts.config import NODES_FILE, EDGES_FILE

In [3]:
# Load data

nodes = pd.read_csv(NODES_FILE)  
# columns: user_id, username, is_agent

edges = pd.read_csv(EDGES_FILE)  
# columns: source_id, target_id, timestamp, tweet_id, type

# Filter by time window

start_date = "2025-08-01"  # first date in edges.csv
end_date = "2025-12-31"    # date of project submission :)

edges['timestamp'] = pd.to_datetime(edges['timestamp'])
edges_window = edges[
    (edges['timestamp'] >= start_date) &
    (edges['timestamp'] <= end_date)
].copy()

# Map user_id to igraph vertex index

user_id_to_index = {
    uid: idx for idx, uid in enumerate(nodes['user_id'])
}

edges_window['source_idx'] = edges_window['source_id'].map(user_id_to_index)
edges_window['target_idx'] = edges_window['target_id'].map(user_id_to_index)

# Drop edges involving users not present in the node list
edges_window = edges_window.dropna(subset=['source_idx', 'target_idx'])

edges_window[['source_idx', 'target_idx']] = (
    edges_window[['source_idx', 'target_idx']].astype(int)
)

# Aggregate interactions
# One edge = one (source, target) pair


edge_weights = (
    edges_window
    .groupby(['source_idx', 'target_idx'])
    .size()
    .reset_index(name='weight')
)

# Create directed, weighted igraph graph

G = ig.Graph(directed=True)

# Add all nodes
G.add_vertices(len(nodes))

G.vs['user_id'] = nodes['user_id'].tolist()
G.vs['username'] = nodes['username'].tolist()
G.vs['is_agent'] = nodes['is_agent'].tolist()

# Add edges
G.add_edges(
    list(zip(edge_weights['source_idx'], edge_weights['target_idx']))
)

# Add weights
G.es['weight'] = edge_weights['weight'].tolist()

# Descriptive statistics (Question 5 output)

num_nodes = G.vcount()
num_edges = G.ecount()

num_agents = sum(G.vs['is_agent'])
num_humans = num_nodes - num_agents

in_degrees = G.indegree()
out_degrees = G.outdegree()

print("===== GRAPH STATISTICS =====")
print(f"Nodes: {num_nodes}")
print(f"Edges (unique interactions): {num_edges}")
print(f"Humans: {num_humans}")
print(f"Agents: {num_agents}")

print("\nIn-degree statistics:")
print("  Mean:", round(np.mean(in_degrees), 2))
print("  Median:", np.median(in_degrees))
print("  Max:", max(in_degrees))

print("\nOut-degree statistics:")
print("  Mean:", round(np.mean(out_degrees), 2))
print("  Median:", np.median(out_degrees))
print("  Max:", max(out_degrees))


===== GRAPH STATISTICS =====
Nodes: 40775
Edges (unique interactions): 38592
Humans: 25477
Agents: 15298

In-degree statistics:
  Mean: 0.95
  Median: 0.0
  Max: 375

Out-degree statistics:
  Mean: 0.95
  Median: 0.0
  Max: 232


## Question 6

À l’aide de GEPHI ou de R/Python avec igraph, construisez le graphe correspondant à la fenêtre temporelle retenue. Indiquez dans votre rapport le nombre de nœuds, le nombre d’arêtes, la proportion de comptes humains vs agents, ainsi que des statistiques simples sur les degrés entrants et sortants (moyenne, médiane, maximum). Commentez ce que ces chiffres suggèrent sur la taille et la densité de la communauté.

## Question 7

Produisez au moins une visualisation du graphe global dans GEPHI. Précisez l’algorithme de mise en page utilisé et la signification des choix de représentation (par exemple taille des nœuds en fonction du degré entrant, couleur en fonction du type de compte humain vs agent). Intégrez la figure dans votre rapport et interprétez ce que vous observez. Identifiez notamment si certains agents semblent jouer un rôle de hub.

## Question 8

Comparez, de manière descriptive, la distribution de l’engagement local entre humains et agents. Définissez l’engagement local comme un ratio du type « réponses et mentions reçues / nombre de messages émis » (vous pouvez ajuster la formule). Calculez des statistiques de base pour ce ratio par type d’acteur et discutez si les agents semblent sur- ou sous-performer par rapport aux humains.

---
# 4. Communautés, centralisation et fragmentation (socle commun)

L’objectif ici est de détecter des sous-communautés et de mesurer la structure globale du réseau, avec et sans agents.

## Question 9

Appliquez un algorithme de détection de communautés sur votre graphe (par exemple Louvain ou Leiden). Indiquez les paramètres principaux retenus (résolution, nombre d’itérations, etc.) et justifiez ces choix. Donnez le nombre de communautés détectées, leurs tailles (par exemple en indiquant les tailles minimale, maximale et médiane) et la valeur de modularité associée.

## Question 10

Analysez la répartition des agents dans les communautés détectées. Identifiez si certains clusters sont particulièrement « riches » en agents ou, au contraire, quasi exclusivement humains. Discutez ce que cela peut signifier en termes de rôle des agents dans certaines parties du réseau.

## Question 11

Calculez un indicateur de centralisation globale du réseau pour la même fenêtre temporelle, par exemple la centralisation de Freeman sur le degré entrant. Présentez la valeur obtenue, expliquez brièvement la formule utilisée et discutez la signification empirique de cette valeur pour votre réseau Crypto Twitter. Est-ce que quelques nœuds concentrent fortement l’attention ou la distribution semble plus égalitaire ?

## Question 12

Calculez un indicateur de fragmentation globale, par exemple la modularité de la partition détectée ou un indice E–I comparant liens intra-communautés et inter-communautés. Discutez la cohérence de cet indicateur avec la visualisation obtenue. Les communautés vous semblent-elles effectivement bien séparées ou plutôt interconnectées ?

## Question 13

Reprenez les calculs de centralisation et de fragmentation, cette fois en construisant un graphe où vous supprimez les nœuds « agents » et toutes les arêtes qui les impliquent. Comparez quantitativement les indicateurs obtenus dans les deux configurations (avec agents vs humains seuls) et discutez dans quelle mesure ces résultats sont compatibles avec l’hypothèse H2.

---
# 5. Premiers tests d’hypothèses H1 et H2 (socle commun)

Cette partie vise à proposer un premier examen empirique, même simple, des hypothèses H1 et H2.

## Question 14

En vous appuyant sur les résultats de la question 8, discutez dans quelle mesure les agents les plus intégrés (par exemple ceux ayant un degré entrant élevé ou appartenant au cœur du réseau) reçoivent effectivement plus d’engagement local. Proposez au moins un graphique (par exemple un nuage de points) mettant en relation un indicateur d’intégration (degré, centralité) et l’engagement local. Interprétez ce graphique au regard de H1.

## Question 15

À partir de la comparaison entre le réseau complet et le réseau restreint aux humains (question 13), tirez une conclusion provisoire sur H2. Les agents semblent-ils contribuer à la centralisation de l’attention, à la fragmentation du réseau, aux deux, à aucun ? Distinguez clairement ce que vous pouvez affirmer des simples suggestions exploratoires.

---
# 6. Extensions avancées (pour aller plus loin vers un travail de recherche)

Les questions suivantes sont destinées à ceux qui souhaitent approfondir leur analyse. Elles ne sont pas toutes obligatoires, mais traiter plusieurs de ces questions avec rigueur permet d’atteindre une meilleure note. Il est préférable de traiter en profondeur quelques questions avancées plutôt que de répondre superficiellement à toutes.

## 6.1. Analyse temporelle de la structure du réseau

### Question 16 (avancé)

Découpez votre période d’observation en plusieurs fenêtres temporelles successives (par exemple jours, semaines ou mois, selon la taille des données). Pour chaque fenêtre, construisez le graphe d’interactions et calculez au minimum les indicateurs suivants : proportion d’agents dans le réseau, centralisation globale, modularité ou indice de fragmentation. Représentez l’évolution de ces indicateurs au cours du temps dans un graphique. Commentez les co-variations observées, en particulier entre la proportion d’agents et les mesures de centralisation et de fragmentation.

### Question 17 (avancé)

Identifiez visuellement, à l’aide de deux ou trois instantanés de réseau à des dates différentes, si la structure semble se reconfigurer lorsque l’activité des agents augmente ou diminue. Discutez la cohérence de vos observations qualitatives avec les séries d’indicateurs calculés à la question 16.

## 6.2. Relation non linéaire « dose d’agents » – engagement humain (H3)

### Question 18 (avancé)

Construisez, pour chaque utilisateur humain et pour une période donnée (ou en moyenne sur plusieurs périodes), un indicateur de « dose d’agents » défini comme la part de ses interactions impliquant au moins un agent. Précisez clairement comment vous traitez les interactions multiples et la distinction entre interactions envoyées et reçues.

### Question 19 (avancé)

Pour ces mêmes utilisateurs, calculez un indicateur d’engagement humain (par exemple nombre moyen de réponses reçues par tweet). Représentez la relation entre dose d’agents et engagement humain à l’aide d’un nuage de points ou d’un graphique lissé. Indiquez si la relation semble plutôt monotone ou si l’on observe un maximum pour une dose intermédiaire.

### Question 20 (avancé)

Estimez un modèle simple de type engagement = a + b1 × dose + b2 × dose² (en R ou Python). Interprétez les signes et la significativité des coefficients b1 et b2, dans la mesure où les données le permettent. Discutez les limites de cette analyse (par exemple nombre d’observations, hétérogénéité des utilisateurs, variables de contrôle non prises en compte) et indiquez si vos résultats sont compatibles avec H3.

## 6.3. Imitation linguistique et engagement (H1 renforcée)

### Question 21 (avancé)

Proposez une opérationnalisation simple de la similarité linguistique entre un agent et les humains de sa communauté. Par exemple, vous pouvez construire pour chaque compte un vocabulaire de hashtags ou de mots fréquents, et mesurer une similarité de type Jaccard entre le vocabulaire de l’agent et celui des humains du même cluster. Décrivez précisément votre procédure.

### Question 22 (avancé)

Pour les agents, mettez en relation la similarité linguistique ainsi calculée et l’engagement local qu’ils reçoivent. Représentez cette relation graphiquement et discutez si les agents qui « parlent comme les humains » semblent effectivement mieux intégrés et plus engagés. Reliez votre interprétation à l’idée d’imitation linguistique dans le cadre théorique.

## 6.4. Robustesse des résultats de structure de réseau

### Question 23 (avancé)

Testez la robustesse de vos conclusions sur la centralisation et la fragmentation en faisant varier au moins deux éléments du design de votre graphe. Par exemple, modifiez le seuil minimal de poids pour conserver une arête, ou restreignez le graphe à certains types d’interactions (mentions seules vs mentions + réponses + retweets). Recalculez les indicateurs clés et discutez si vos conclusions qualitatives restent stables.

### Question 24 (avancé)

Comparez les partitions de communautés obtenues avec au moins deux algorithmes différents (par exemple Louvain et Leiden). Présentez les valeurs de modularité et les caractéristiques principales des partitions. Discutez en quoi les résultats convergent ou divergent, et ce que cela implique pour l’interprétation de la fragmentation.

## 6.5. Réflexion méthodologique et causalité

### Question 25 (avancé)

Identifiez les principales menaces à une interprétation causale de vos résultats. Discutez, à partir de votre expérience de ce TP, au moins trois sources potentielles de biais (par exemple sélection des agents, événements exogènes sur la plateforme, erreurs de classification humain vs agent). Pour chacune, indiquez comment un design de recherche plus avancé pourrait tenter de les atténuer.

### Question 26 (avancé)

Proposez, au niveau conceptuel, un design quasi-expérimental ou de panel qui permettrait de tester plus rigoureusement l’impact des agents sur la structure du réseau (par exemple exploitation d’un changement de politique d’étiquetage, d’une purge de comptes, ou d’une introduction contrôlée d’agents dans certaines sous-communautés). Vous n’avez pas besoin de le mettre en œuvre, mais vous devez spécifier l’unité d’analyse, le traitement, le groupe contrôle et les principaux indicateurs.
